# YOLO11

## установка и импорт необходимых зависимостей 

In [ ]:
!pip install ultralytics --q
!pip install albumentations==1.4.14 --q
!pip install -U ipywidgets --q

In [ ]:
import os
import json
import shutil
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import cv2
from ultralytics import YOLO
import traceback

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

## создание рабочих директорий

In [ ]:
!mkdir /kaggle/working/datasets
!mkdir /kaggle/working/datasets/prepared_data
!mkdir /kaggle/working/config

## конвертирование исходного датасета в формат подходящий для YOLO

In [ ]:
def convert_to_yolo_format(bbox, img_width, img_height):
    """Конвертация координат из абсолютных в относительные для YOLO"""
    x_center = (bbox[0] + bbox[2]) / 2 / img_width
    y_center = (bbox[1] + bbox[3]) / 2 / img_height
    width = (bbox[2] - bbox[0]) / img_width
    height = (bbox[3] - bbox[1]) / img_height
    return [x_center, y_center, width, height]

def prepare_dataset():
    """Подготовка датасета"""
    raw_data_dir = Path('/kaggle/input/final-dataset/raw_data')
    output_dir = Path('/kaggle/working/datasets/prepared_data')
    
    print(f"Проверка директорий:")
    print(f"raw_data_dir: {raw_data_dir}")
    print(f"output_dir: {output_dir}")

    # проверяем существование директорий
    if not (raw_data_dir / "images").exists():
        raise ValueError(f"Директория с изображениями не найдена: {raw_data_dir / 'images'}")
    if not (raw_data_dir / "jsons").exists():
        raise ValueError(f"Директория с JSON файлами не найдена: {raw_data_dir / 'jsons'}")

    # создаем необходимые директории
    images_dir = output_dir / "images"
    labels_dir = output_dir / "labels"
    images_dir.mkdir(parents=True, exist_ok=True)
    labels_dir.mkdir(parents=True, exist_ok=True)

    # словарь для маппинга классов
    class_map = {
        "title": 0, "paragraph": 1, "table": 2, "picture": 3,
        "table_signature": 4, "picture_signature": 5, "numbered_list": 6,
        "marked_list": 7, "header": 8, "footer": 9, "footnote": 10,
        "formula": 11
    }

    # собираем пары файлов
    valid_pairs = []
    image_files = list(Path(raw_data_dir / "images").glob("*.png"))
    print(f"\nНайдено {len(image_files)} PNG файлов")
    
    cnt = 0
    for img_path in image_files:
        json_path = raw_data_dir / "jsons" / f"{img_path.stem}.json"
        
        if not json_path.exists():
            print(f"Пропускаем {img_path.name}: нет соответствующего JSON файла")
            continue

        try:
            # проверяем, что изображение читается
            img = cv2.imread(str(img_path))
            if img is None:
                print(f"Пропускаем {img_path.name}: невозможно прочитать изображение")
                continue

            # копируем изображение
            new_img_path = images_dir / img_path.name
            shutil.copy(str(img_path), str(new_img_path))

            # читаем JSON
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # создаем YOLO-формат аннотаций
            label_content = []
            img_width = data['image_width']
            img_height = data['image_height']

            for class_name, boxes in data.items():
                if class_name in class_map and isinstance(boxes, list) and boxes:
                    class_id = class_map[class_name]
                    for bbox in boxes:
                        yolo_bbox = convert_to_yolo_format(bbox, img_width, img_height)
                        label_content.append(f"{class_id} {' '.join(map(str, yolo_bbox))}")

            # сохраняем файл с аннотациями
            label_file = labels_dir / f"{img_path.stem}.txt"
            with open(label_file, 'w') as f:
                f.write('\n'.join(label_content))

            valid_pairs.append(img_path.stem)
            cnt += 1
            
        except Exception as e:
            print(f"Ошибка при обработке {img_path.name}: {str(e)}")
            continue

    print(f"\nИтоги обработки:")
    print(f"Всего найдено изображений: {len(image_files)}")
    print(f"Успешно обработано пар: {len(valid_pairs)}")

    if not valid_pairs:
        raise ValueError("Не найдено валидных пар изображение-разметка!")

    # создаем train/val split (80/20)
    np.random.shuffle(valid_pairs)
    split_idx = int(len(valid_pairs) * 0.8)
    train_pairs = valid_pairs[:split_idx]
    val_pairs = valid_pairs[split_idx:]

    # сохраняем списки train/val с полными путями
    with open(output_dir / "train.txt", 'w') as f:
        f.write('\n'.join(str(images_dir / f"{name}.png") for name in train_pairs))
    
    with open(output_dir / "val.txt", 'w') as f:
        f.write('\n'.join(str(images_dir / f"{name}.png") for name in val_pairs))

    print(f"\nСоздание файлов со списками:")
    print(f"train.txt: {len(train_pairs)} образцов")
    print(f"val.txt: {len(val_pairs)} образцов")
    
    return len(valid_pairs)

def main():
    # подготовка данных
    print("Подготовка данных...")
    num_samples = prepare_dataset()
    output_dir = Path('/kaggle/working/datasets/prepared_data')

    # создаем конфигурационный файл
    config_dir = Path('/kaggle/working/config')
    config_dir.mkdir(exist_ok=True)
    
    config_content = f"""
                path: {str(output_dir)}
                train: {str(output_dir / 'train.txt')}
                val: {str(output_dir / 'val.txt')}
                
                nc: 12
                names: ['title', 'paragraph', 'table', 'picture', 'table_signature', 'picture_signature', 
                        'numbered_list', 'marked_list', 'header', 'footer', 'footnote', 'formula']
                """
    
    config_path = config_dir / "doclayout.yaml"
    with open(config_path, 'w') as f:
        f.write(config_content)

if __name__ == "__main__":
    main()

## добавление инструментов визуализации и контроля

In [ ]:
!pip install wandb --q
!pip install comet_ml --q

## устранение конфликтов

In [ ]:
!pip uninstall ray[tune] -y -q

In [ ]:
import comet_ml
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("COMET")
secret_value_1 = user_secrets.get_secret("WANDB")

In [ ]:
comet_ml.init(api_key=secret_value_0)
wandb.login(key=secret_value_1)

## обучение модели

In [ ]:
def main():
    # определяем пути
    project_root = Path('/kaggle/working')
    raw_data_dir = Path('/kaggle/input/final-dataset/raw_data')
    output_dir = Path('/kaggle/working/datasets/prepared_data')

    # Создаем конфигурационный файл
    config_dir = Path('/kaggle/working/config')
    config_dir.mkdir(exist_ok=True)
    
    config_content = f"""
                    path: {str(output_dir)}
                    train: {str(output_dir / 'train.txt')}
                    val: {str(output_dir / 'val.txt')}
                    
                    nc: 12
                    names: ['title', 'paragraph', 'table', 'picture', 'table_signature', 'picture_signature', 
                            'numbered_list', 'marked_list', 'header', 'footer', 'footnote', 'formula']
                    """
    
    config_path = config_dir / "doclayout.yaml"
    with open(config_path, 'w') as f:
        f.write(config_content)

    model = YOLO('yolo11l.pt', verbose=True)
    
    print("Запуск обучения...")
    results = model.train(
        data=str(config_path),
        epochs=50,
        imgsz=640,
        batch=8,
        workers=2,
        project='DocLayout YOLO',
        name="shok",
        save_period=10,
        exist_ok=True,
        amp=False,
        cache=False,
        pretrained=True,
        resume=False,
        verbose=True,
        optimizer="AdamW",
        lr0=0.001,
        lrf=0.01,
        momentum=0.935,
        weight_decay=0.005,
        warmup_epochs=3.0,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        plots=True,
        save=True,
        multi_scale=False,
        dropout=0.1,
        augment=False,
        degrees=3.0,
        hsv_h=0.1,
        hsv_s=0.1, 
        hsv_v=0.1, 
        mosaic=0.0,
        flipud=0.0, 
        fliplr=0.0
    )

    return results

if __name__ == "__main__":
    try:
        # очистка CUDA кэша
        torch.cuda.empty_cache()
        
        # установка переменных окружения для оптимизации памяти
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:1024'
        
        results = main()
        print("Обучение успешно завершено")
    except Exception as e:
        print(f"Произошла ошибка: {e}")
        print("Полный стек ошибки:")
        print(traceback.format_exc())